In [61]:
import csv
import re
import string
from nltk import sent_tokenize
with open('train_bodies.csv',encoding='latin1') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    articles={}
    for i in spamreader:
        articles[i[0]]=i[1]
del articles['Body ID']  
def clean_data(body):
    punctuations = string.punctuation + '—' + '’' + '…' + '‘' + '–' + '”' + '“'
    regex = re.compile('[%s]' % re.escape(punctuations))
    clean_text=''
    for sentence in sent_tokenize(body):
        sentence=regex.sub('', sentence)
        sentence=re.sub(r"\n", " ", sentence)
        sentence = re.sub(r"i'm", "i am", sentence)
        sentence = re.sub(r"he's", "he is", sentence)
        sentence = re.sub(r"she's", "she is", sentence)
        sentence = re.sub(r"it's", "it is", sentence)
        sentence = re.sub(r"that's", "that is", sentence)
        sentence = re.sub(r"what's", "what is", sentence)
        sentence = re.sub(r"where's", "where is", sentence)
        sentence = re.sub(r"how's", "how is", sentence)
        sentence = re.sub(r"\'ll", " will", sentence)
        sentence = re.sub(r"\'ve", " have", sentence)
        sentence = re.sub(r"\'re", " are", sentence)
        sentence = re.sub(r"\'d", " would", sentence)
        sentence = re.sub(r"\'re", " are", sentence)
        sentence = re.sub(r"won't", "will not", sentence)
        sentence = re.sub(r"can't", "cannot", sentence)
        sentence = re.sub(r"n't", " not", sentence)
        sentence = re.sub(r"n'", "ng", sentence)
        sentence = re.sub(r"'bout", "about", sentence)
        sentence = re.sub(r"'til", "until", sentence)
        sentence = re.sub(' +',' ',sentence)
        clean_text=clean_text+' '+sentence
    return clean_text
    #print(clean_text)

 
clean_articles={}
for i in articles:
    clean_articles[int(i)]=clean_data(articles[i])

print(clean_articles[0])
clean_articles_list=[]
single_corpus=''
for i in clean_articles:
    clean_articles_list.append(clean_articles[i])
    single_corpus=single_corpus+' '+clean_articles[i]


 A small meteorite crashed into a wooded area in Nicaraguas capital of Managua overnight the government said Sunday Residents reported hearing a mysterious boom that left a 16foot deep crater near the citys airport the Associated Press reports Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a relatively small meteorite that appears to have come off an asteroid that was passing close to Earth Housesized asteroid 2014 RC which measured 60 feet in diameter skimmed the Earth this weekend ABC News reports Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet said Humberto Saballos a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee He said it is still not clear if the meteorite disintegrated or was buried Humberto Garcia of the Astronomy Center at

In [67]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
vocab=set(single_corpus.split())
vocab = [w for w in vocab if not w in stop_words]
print(vocab[1:20])

['confirmationâ\x80\x9d', 'weirdly', 'vigorous', 'antiMuslim', 'orchestrating', 'requesting', 'surpassing', 'apiece', 'Shebab', 'Keith182', 'Obviously', '20413', 'tanir', '4m', 'dont', 'Guerrero', 'archrival', 'spitting', 'momentum']


In [31]:
#get stances
with open('train_stances.csv',encoding='latin1') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    stances={}
    title={}
    for i in spamreader:
        #print(i[1])
        stances[i[1]]=i[2]
        title[i[1]]=i[0]
#print(stances['0'])
del stances['Body ID']
del title['Body ID']
stances = {int(k):v for k,v in stances.items()}
title = {int(k):v for k,v in title.items()}

final_data=[]
for i in clean_articles:
    final_data.append([i,clean_articles[i],stances[i]])
#print(final_data[1])
#print(stances)
#print(clean_articles[4])

 Last week we hinted at what was to come as Ebola fears spread across America Today we get confirmation As The Daily Caller reports one passenger at Dulles International Airport outside Washington DC is apparently not taking any chances A female passenger dressed in a hazmat suit complete with a full body gown mask and gloves was spotted Wednesday waiting for a flight at the airport Source The Daily Caller We particularly liked the JCPenney bag maybe thats a new business line for the bankrupt retailer On a side note try Halloween stores if you need a HazMat suit in a hurry


In [68]:

#corpus = [   'This is the first document.',     'This is the second second document.',     'And the third one.',     'Is this the first document?',]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=vocab)
t=vectorizer.fit_transform(clean_articles_list)
vect_articles=t.toarray()
print(vect_articles)
y_ids=[]
for i in clean_articles:
    y_ids.append(i)
stance_rearranged=[]
for i in y_ids:
    stance_rearranged.append(stances[i])
#print(stance_rearranged)
y_stance_label=[]
for s in stance_rearranged:
    if s=='unrelated':
        y_stance_label.append(1)
    else :
        y_stance_label.append(0)
        
print(y_stance_label)   

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 

In [69]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=True, C = 1e15)
clf.fit(vect_articles, y_stance_label)

print(clf.intercept_, clf.coef_)
#print(weights)
print('Accuracy from sk-learn: {0}'.format(clf.score(vect_articles, y_stance_label)))



[4.16493188] [[ 0.          0.         -1.58328394 ...  0.          5.43633955
   0.        ]]
Accuracy from sk-learn: 0.9952465834818776


For testing

In [80]:
with open('competition_test_bodies.csv',encoding='latin1') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    articles_test={}
    for i in spamreader:
        articles_test[i[0]]=i[1]
del articles_test['Body ID']  
def clean_data(body):
    punctuations = string.punctuation + '—' + '’' + '…' + '‘' + '–' + '”' + '“'
    regex = re.compile('[%s]' % re.escape(punctuations))
    clean_text=''
    for sentence in sent_tokenize(body):
        sentence=regex.sub('', sentence)
        sentence=re.sub(r"\n", " ", sentence)
        sentence = re.sub(r"i'm", "i am", sentence)
        sentence = re.sub(r"he's", "he is", sentence)
        sentence = re.sub(r"she's", "she is", sentence)
        sentence = re.sub(r"it's", "it is", sentence)
        sentence = re.sub(r"that's", "that is", sentence)
        sentence = re.sub(r"what's", "what is", sentence)
        sentence = re.sub(r"where's", "where is", sentence)
        sentence = re.sub(r"how's", "how is", sentence)
        sentence = re.sub(r"\'ll", " will", sentence)
        sentence = re.sub(r"\'ve", " have", sentence)
        sentence = re.sub(r"\'re", " are", sentence)
        sentence = re.sub(r"\'d", " would", sentence)
        sentence = re.sub(r"\'re", " are", sentence)
        sentence = re.sub(r"won't", "will not", sentence)
        sentence = re.sub(r"can't", "cannot", sentence)
        sentence = re.sub(r"n't", " not", sentence)
        sentence = re.sub(r"n'", "ng", sentence)
        sentence = re.sub(r"'bout", "about", sentence)
        sentence = re.sub(r"'til", "until", sentence)
        sentence = re.sub(' +',' ',sentence)
        clean_text=clean_text+' '+sentence
    return clean_text
    #print(clean_text)

 
clean_articles_test={}
for i in articles_test:
    clean_articles_test[int(i)]=clean_data(articles_test[i])

#print(clean_articles[0])
clean_articles_list_test=[]
for i in clean_articles_test:
    clean_articles_list_test.append(clean_articles_test[i])

t=vectorizer.fit_transform(clean_articles_list_test)
vect_articles_test=t.toarray()
print(vect_articles_test)


print(clf.predict(X=vect_articles_test))
#print('Accuracy from sk-learn: {0}'.format(clf.score(vect_articles_test, y_stance_label)))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[1 1 0 0 0 1 1 0 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0
 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 0
 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0
 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0
 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1
 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 1 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0
 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0
 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 1
 0 0 0 0 1 0 0 0 1 